# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [3]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [6]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [19]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [20]:
quantity = data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})
quantity

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [21]:
prodCust = pd.pivot_table(quantity, index=['ProductName'], columns=['CustomerID'], fill_value=0)

In [22]:
prodCust

Quantity                                \
CustomerID                           33    200   264   356   412   464     
ProductName                                                                
Anchovy Paste - 56 G Tube                0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp        0     0     0     0     0     0   
Appetizer - Mushroom Tart                0     0     0     0     0     0   
Appetizer - Sausage Rolls                0     0     0     0     0     0   
Apricots - Dried                         1     0     0     0     1     0   
...                                    ...   ...   ...   ...   ...   ...   
Yeast Dry - Fermipan                     0     0     0     0     0     0   
Yoghurt Tubes                            0     0     0     0     0     0   
Yogurt - Blueberry, 175 Gr               0     1     0     0     0     0   
Yogurt - French Vanilla                  1     0     0     1     0     0   
Zucchini - Yellow                        0     0     0     0     0     0   

                                                           ...              \
CustomerID                        477   639   649   669    ... 97697 97753   
ProductName                                                ...               
Anchovy Paste - 56 G Tube             0     1     0     0  ...     0    25   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0  ...    25    25   
Appetizer - Mushroom Tart             0     1     0     0  ...    25     0   
Appetizer - Sausage Rolls             0     0     0     0  ...     0     0   
Apricots - Dried                      0     0     0     0  ...     0    25   
...                                 ...   ...   ...   ...  ...   ...   ...   
Yeast Dry - Fermipan                  0     0     0     0  ...     0     0   
Yoghurt Tubes                         0     0     2     0  ...     0     0   
Yogurt - Blueberry, 175 Gr            0     0     0     0  ...    25     0   
Yogurt - French Vanilla               2     0     0     1  ...     0     0   
Zucchini - Yellow                     0     0     0     0  ...     0     0   

                                                                             \
CustomerID                        97769 97793 97900 97928 98069 98159 98185   
ProductName                                                                   
Anchovy Paste - 56 G Tube             0     0     0     0     0     0     0   
Appetizer - Mini Egg Roll, Shrimp     0     0     0     0     0     0     0   
Appetizer - Mushroom Tart             0     0     0     0     0     0    25   
Appetizer - Sausage Rolls             0     0    25    25    25     0    25   
Apricots - Dried                      0     0     0     0     0     0     0   
...                                 ...   ...   ...   ...   ...   ...   ...   
Yeast Dry - Fermipan                  0    25     0     0     0     0     0   
Yoghurt Tubes                         0     0     0     0     0    25     0   
Yogurt - Blueberry, 175 Gr            0    25     0     0     0     0     0   
Yogurt - French Vanilla              25     0     0     0     0     0     0   
Zucchini - Yellow                     0     0     0     0     0     0     0   

                                         
CustomerID                        98200  
ProductName                              
Anchovy Paste - 56 G Tube             0  
Appetizer - Mini Egg Roll, Shrimp     0  
Appetizer - Mushroom Tart             0  
Appetizer - Sausage Rolls             0  
Apricots - Dried                      0  
...                                 ...  
Yeast Dry - Fermipan                  0  
Yoghurt Tubes                         0  
Yogurt - Blueberry, 175 Gr            0  
Yogurt - French Vanilla              25  
Zucchini - Yellow                     0  

[452 rows x 1000 columns]

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.